In [1]:
import leafmap
from samgeo import SamGeo2, raster_to_vector, regularize

In [4]:
m = leafmap.Map(center=[29.6768, -95.3692], zoom=19)
m.add_basemap("SATELLITE")
m

Map(center=[29.6768, -95.3692], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'z…

In [5]:
image = '/home/jgillan/Documents/pima_aerial/pictometry2.tif'


In [6]:
m.layers[-1].visible = False
m.add_raster(image, layer_name="Image")
m

Map(bottom=55514121.0, center=[32.071846, -110.9369725], controls=(ZoomControl(options=['position', 'zoom_in_t…

In [5]:
sam2 = SamGeo2(
        model_id="sam2-hiera-large",
        device="cuda", 
        automatic=False,
        crop_n_layers=2,
        crop_overlap_ratio=0.1
)

In [6]:
sam2.set_image(image)

In [ ]:
m = sam2.show_map()
m

In [7]:
import os
import numpy as np
from PIL import Image
from samgeo import SamGeo2

import torch
import gc

In [8]:


# --------------------------------
# 1. Helper function to split large images into overlapping tiles
def split_image(image, tile_size=2048, overlap=512):
    width, height = image.size
    stride = tile_size - overlap
    tiles = []
    coords = []
    
    for y in range(0, height, stride):
        for x in range(0, width, stride):
            box = (x, y, min(x + tile_size, width), min(y + tile_size, height))
            tile = image.crop(box)
            tiles.append(tile)
            coords.append((x, y))
    return tiles, coords

# --------------------------------
# 2. Load your large image
large_image_path = "/home/jgillan/Documents/pima_aerial/pictometry2.tif"
image = Image.open(large_image_path).convert("RGB")

# --------------------------------
# 3. Split image into tiles
tile_size = 2048    # or 1024 if memory tight
overlap = 200       # adjust overlap to avoid cutting objects
tiles, coords = split_image(image, tile_size=tile_size, overlap=overlap)

print(f"Split image into {len(tiles)} tiles.")

# --------------------------------
# 4. Initialize SAMGeo2 model
sam = SamGeo2(
    model_id="sam2-hiera-large",
    device="cuda",
    automatic=True,   # yes, use automatic mask generation on each tile
)

# --------------------------------
# 5. Predict masks on each tile
all_masks = []

for idx, tile in enumerate(tiles):
    print(f"Processing tile {idx+1}/{len(tiles)}...")
    
    # Convert PIL image tile to numpy array
    tile_np = np.array(tile)
    
    try:
        masks = sam.generate(tile_np)
        all_masks.append((coords[idx], masks))
    except RuntimeError as e:
        print(f"OOM at tile {idx}. Trying to recover...")
        torch.cuda.empty_cache()
        gc.collect()

# --------------------------------
# 6. Done! Now you have masks for each tile.
# (Optional) You can stitch them together or save them individually.
print(f"Finished predicting {len(all_masks)} tile masks.")


Split image into 36 tiles.
Processing tile 1/36...
Processing tile 2/36...
Processing tile 3/36...
Processing tile 4/36...
Processing tile 5/36...
Processing tile 6/36...
Processing tile 7/36...
Processing tile 8/36...
Processing tile 9/36...
Processing tile 10/36...
Processing tile 11/36...
Processing tile 12/36...
Processing tile 13/36...
Processing tile 14/36...
Processing tile 15/36...
Processing tile 16/36...
Processing tile 17/36...
Processing tile 18/36...
Processing tile 19/36...
Processing tile 20/36...
Processing tile 21/36...
Processing tile 22/36...
Processing tile 23/36...
Processing tile 24/36...
Processing tile 25/36...
Processing tile 26/36...
Processing tile 27/36...
Processing tile 28/36...
Processing tile 29/36...
Processing tile 30/36...
Processing tile 31/36...
Processing tile 32/36...
Processing tile 33/36...
Processing tile 34/36...
Processing tile 35/36...
Processing tile 36/36...
Finished predicting 36 tile masks.


In [9]:
import numpy as np

# Create full-sized mask canvas
stitched_mask = np.zeros((image.height, image.width), dtype=np.uint8)

for (x_offset, y_offset), masks in all_masks:
    if masks is None:
        continue
    for mask in masks:
        seg = mask['segmentation']
        h, w = seg.shape
        stitched_mask[y_offset:y_offset+h, x_offset:x_offset+w] = np.maximum(
            stitched_mask[y_offset:y_offset+h, x_offset:x_offset+w],
            seg.astype(np.uint8)
        )


In [10]:
from PIL import Image

downscale = 8  # try 4 or 8 depending on original image size
small_mask = stitched_mask[::downscale, ::downscale]

# Convert to transparent red RGBA image
rgba = np.zeros((small_mask.shape[0], small_mask.shape[1], 4), dtype=np.uint8)
rgba[small_mask > 0] = [255, 0, 0, 150]  # Red + alpha
Image.fromarray(rgba).save("stitched_mask_overlay.png")


In [11]:
import rasterio
from rasterio.warp import transform_bounds

with rasterio.open(large_image_path) as src:
    bounds = src.bounds  # (left, bottom, right, top)
    crs = src.crs
    bounds_latlon = transform_bounds(crs, "EPSG:4326", *bounds)

west, south, east, north = bounds_latlon

In [12]:
import leafmap

m = leafmap.Map(center=[(south + north) / 2, (west + east) / 2], zoom=16)
m.add_basemap("SATELLITE")

# Add the transparent image overlay
m.add_image(
    "stitched_mask_overlay.png",
    bounds=[[south, west], [north, east]],
    opacity=0.5,
    name="SAM Segmentation"
)

m.add_layer_control()
m


Map(center=[32.071845715852064, -110.93697267636854], controls=(ZoomControl(options=['position', 'zoom_in_text…

In [13]:
import numpy as np
from PIL import Image

# Assume `small_mask` is the downscaled binary mask
# where 0 = background, 1 = object

# Create an empty RGBA array
rgba_arr = np.zeros((small_mask.shape[0], small_mask.shape[1], 4), dtype=np.uint8)

# Set object pixels to red with transparency
rgba_arr[small_mask > 0] = [255, 0, 0, 150]  # red + semi-transparent

# Set background pixels to fully transparent
# (Already zeros from np.zeros, just making it explicit)

# Save as PNG
Image.fromarray(rgba_arr).save("stitched_mask_rgba_clean.png")


In [14]:
import leafmap

m = leafmap.Map(center=[(south + north)/2, (west + east)/2], zoom=16)
m.add_basemap("SATELLITE")

m.add_image(
    "stitched_mask_rgba_clean.png",
    bounds=[[south, west], [north, east]],
    opacity=1.0,  # use full opacity because alpha is already encoded
    name="Segmented Overlay"
)

m.add_layer_control()
m


Map(center=[32.071845715852064, -110.93697267636854], controls=(ZoomControl(options=['position', 'zoom_in_text…

In [15]:
# Downscale stitched mask for testing
downscale_factor = 16  # very aggressive downscale

tiny_mask = stitched_mask[::downscale_factor, ::downscale_factor]

rgba = np.zeros((tiny_mask.shape[0], tiny_mask.shape[1], 4), dtype=np.uint8)
rgba[tiny_mask > 0] = [255, 0, 0, 150]  # red, semi-transparent

Image.fromarray(rgba).save("tiny_overlay.png")


In [17]:
import leafmap

m = leafmap.Map(center=[(south + north)/2, (west + east)/2], zoom=17)
m.add_basemap("SATELLITE")

m.add_image(
    "tiny_overlay.png",
    bounds=[[south, west], [north, east]],
    opacity=1.0,
    name="Tiny Overlay"
)

m.add_layer_control()
m


Map(center=[32.071845715852064, -110.93697267636854], controls=(ZoomControl(options=['position', 'zoom_in_text…